<a href="https://colab.research.google.com/github/lykskai/HodgkinAvatar/blob/text-model/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Install Required Libraries

In [1]:
!pip install transformers datasets torch pdfplumber wikipedia-api
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=6ccd8e1cea0fcb60f7bd16f7c6a880fa3d2a901f644201698cc4bc568cd7427c

2) Importing the necessary libraries for our code

In [2]:
from transformers import pipeline, Trainer, TrainingArguments
from datasets import Dataset
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import pdfplumber
import wikipediaapi
import torch
import json

** For uploading and extracting files!

In [ ]:
# Upload a file manually
uploaded = files.upload()

# Extract full text from the uploaded file
for filename in uploaded.keys():
    print(f"Processing file: {filename}")
    with pdfplumber.open(filename) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

    # Save extracted text for manual editing
    output_file = filename.replace(".pdf", "_extracted.txt")
    with open(output_file, "w") as f:
        f.write(full_text)
        print(f"Text extracted and saved to {output_file}")

### b. Manually Edit Extracted Text

Saving tf9332901032.pdf to tf9332901032.pdf
Processing file: tf9332901032.pdf
Text extracted and saved to tf9332901032_extracted.txt


** For downloading extracted files!

In [ ]:
from google.colab import files

files.download("tf9332901032_extracted.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

3) Upload and Process Text file

In [4]:
# Upload the text file
from google.colab import files ##

uploaded = files.upload()

# Read the uploaded file
for filename in uploaded.keys():
    if filename == "4articles-dch.txt":
        with open(filename, "r") as f:
            combined_text = f.read()

print("Combined text loaded successfully!")

Saving 4articles-dch.txt to 4articles-dch.txt
Combined text loaded successfully!


4) Create a dataset, convert the combined text into a Hugging Face Dataset:


In [ ]:
lines = combined_text.split("\n")  # Split into lines or entries

dataset = Dataset.from_dict({"text": lines})

5)  Tokenize the Dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

6) Split the dataset

In [ ]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

7) Incorporate RAG for retrieval

In [13]:
# a. Prepare Passages for Retrieval
passages = [
    {"text": combined_text}  # From uploaded dataset
]

# Save passages to a JSON file
with open("passages.json", "w") as f:
    json.dump(passages, f)

# b. Load the RAG Model and Retriever
rag_model_name = "facebook/rag-sequence-base"
rag_tokenizer = RagTokenizer.from_pretrained(rag_model_name)
rag_retriever = RagRetriever.from_pretrained(
    rag_model_name,
    index_name="custom",
    passages_path="./passages.json"
)
rag_model = RagSequenceForGeneration.from_pretrained(rag_model_name, retriever=rag_retriever)

# c. Test the RAG Pipeline
input_question = "What did Dorothy Hodgkin study?"
inputs = rag_tokenizer(input_question, return_tensors="pt")
outputs = rag_model.generate(input_ids=inputs["input_ids"])
print(rag_tokenizer.decode(outputs[0], skip_special_tokens=True))

SyntaxError: invalid syntax (<ipython-input-13-c3162c91b285>, line 1)

8) define the data collator

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

9) Set Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=200,
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
)

10) Fine tune LLaMA for style

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer

# Load tokenizer and model
model_id = "meta-llama/Meta-Llama-3-8B"
model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(model_id)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the Model
trainer.train()

# Test the model using a text-generation pipeline
from transformers import pipeline

generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
print(generation_pipeline("Hey, how are you doing today?", max_length=50))

11) Save the Fine-Tuned Model

In [ ]:
trainer.save_model("./fine_tuned_llama")

3) Load the pretrained LLaMA model and tokenizer

In [ ]:
# Load the LLaMA 3 model using Hugging Face
model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = pipeline("text-generation", model=model_id,
                    model_kwargs={"torch_dtype": torch.bfloat16},
                    device_map="auto")

# Test the pipeline
print(pipeline("Hey, how are you doing today?"))

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Hey, how are you doing today? I hope you are having a great day and that you are ready for a fun and exciting blog post! Today I have a review of the new Dior Addict Lip Glow Colour Reviver Balm in the shade 002 Pink. This lip balm is a new release from Dior and I have been dying to get my hands on it since I first saw it on Instagram. I have been a huge fan of the original Dior Addict Lip Glow for years and years now and it is one of my all time favourite lip products, so I had to try the new balm!\nThe Dior Addict Lip Glow Colour Reviver Balm in 002 Pink is a lip balm that is formulated with a complex of hyaluronic acid and mango butter to provide your lips with a boost of hydration. It is also enriched with a colour pigments that are designed to react with your natural pH to create a custom colour that is perfect for you. The lip balm is also infused with rose oil to provide a subtle scent and it is formulated without parabens and mineral oils.\nThe lip balm com